In [1023]:
import pandas as pd
import matplotlib as plt
import sklearn as sk

In [1024]:
df = pd.read_csv('train.csv')

### As we see, There is a lot of useless Data, such as Name, Sex, Etc...

### Pre-Processing :

In [1025]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [1026]:
def change_age(age):
    if(age<=10):
        return 'Child'
    elif(10<age<=20):
       return 'teenager'
    elif(20<age<=60):
       return 'Adult'
    elif(age>60):
        return 'Ederly'
        

In [1027]:
def add_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    if pd.isnull(Age):
        return int(df[df["Pclass"] == Pclass]["Age"].mean())
    else:
        return Age

In [1028]:
df["Age"] = df[["Age", "Pclass"]].apply(add_age,axis=1)

In [1029]:
df['Age']=df['Age'].apply(change_age)


In [1030]:
df = df.drop(["Name","Ticket","PassengerId","Fare","Cabin","Parch","SibSp","Embarked"], axis=1)

In [1031]:
replace_dict = {'female': 1, 'male': 0}
df.replace(replace_dict, inplace=True)

In [1032]:
df.fillna(method='ffill', inplace=True)

In [1033]:
df

,Survived,Pclass,Sex,Age
0,0,3,0,Adult
1,1,1,1,Adult
2,1,3,1,Adult
3,1,1,1,Adult
4,0,3,0,Adult
...,...,...,...,...
886,0,2,0,Adult
887,1,1,1,teenager
888,0,3,1,Adult
889,1,1,0,Adult


*Checking For missing values :*

In [1034]:
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
dtype: int64

## Training The data

Let's divite It !

## Random Forest Model

### There is always a peak n_splits where we obtain the most accurate data, let's find out !

In [1035]:
from sklearn.preprocessing import OneHotEncoder

In [1036]:
encoder = OneHotEncoder()

In [1037]:
encoded_age=encoder.fit_transform(df[['Age']])

In [1038]:
encoded_age = pd.DataFrame(encoded_age.toarray(), columns=encoder.get_feature_names_out(['Age']))

In [1039]:
encoded_age

,Age_Adult,Age_Child,Age_Ederly,Age_teenager
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0
...,...,...,...,...
886,1.0,0.0,0.0,0.0
887,0.0,0.0,0.0,1.0
888,1.0,0.0,0.0,0.0
889,1.0,0.0,0.0,0.0


In [1040]:
df = pd.concat([df, encoded_age], axis=1)

In [1041]:
df = df.drop(['Age'], axis=1)

In [1042]:
df

,Survived,Pclass,Sex,Age_Adult,Age_Child,Age_Ederly,Age_teenager
0,0,3,0,1.0,0.0,0.0,0.0
1,1,1,1,1.0,0.0,0.0,0.0
2,1,3,1,1.0,0.0,0.0,0.0
3,1,1,1,1.0,0.0,0.0,0.0
4,0,3,0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
886,0,2,0,1.0,0.0,0.0,0.0
887,1,1,1,0.0,0.0,0.0,1.0
888,0,3,1,1.0,0.0,0.0,0.0
889,1,1,0,1.0,0.0,0.0,0.0


In [1043]:
X = df.drop(["Survived"], axis=1).values
y = df["Survived"].values

In [1044]:
df

,Survived,Pclass,Sex,Age_Adult,Age_Child,Age_Ederly,Age_teenager
0,0,3,0,1.0,0.0,0.0,0.0
1,1,1,1,1.0,0.0,0.0,0.0
2,1,3,1,1.0,0.0,0.0,0.0
3,1,1,1,1.0,0.0,0.0,0.0
4,0,3,0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
886,0,2,0,1.0,0.0,0.0,0.0
887,1,1,1,0.0,0.0,0.0,1.0
888,0,3,1,1.0,0.0,0.0,0.0
889,1,1,0,1.0,0.0,0.0,0.0


In [1045]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
model = RandomForestClassifier()

##### Looks like the best score is at n_splits = 15

In [1046]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

##### Looks like the best score is at n_splits = 60

The best model here seems to be Random Forest at n_splits = 15 !

## Final test

In [1047]:
test_data = pd.read_csv('test.csv')

In [1048]:
def add_age2(cols):
    Age = cols[0]
    Pclass = cols[1]
    if pd.isnull(Age):
        return int(test_data[test_data["Pclass"] == Pclass]["Age"].mean())
    else:
        return Age

In [1049]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [1050]:

test_data["Age"] = test_data[["Age", "Pclass"]].apply(add_age2,axis=1)
test_data['Age']=test_data['Age'].apply(change_age)
replace_dict = {'female': 1, 'male': 0}
test_data.replace(replace_dict, inplace=True)
test_data.fillna(method='ffill', inplace=True)
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",0,Adult,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,Adult,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",0,Ederly,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",0,Adult,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,Adult,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",0,Adult,0,0,A.5. 3236,8.0500,C78,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",1,Adult,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",0,Adult,0,0,SOTON/O.Q. 3101262,7.2500,C105,S
416,1308,3,"Ware, Mr. Frederick",0,Adult,0,0,359309,8.0500,C105,S


In [1051]:
encoded_age2=encoder.fit_transform(test_data[['Age']])
encoded_age2 = pd.DataFrame(encoded_age2.toarray(), columns=encoder.get_feature_names_out(['Age']))

In [1052]:

test_data=pd.concat([test_data, encoded_age2], axis=1)


In [1053]:
model_final = LogisticRegression()
for i in range (2,30):
    test_datascores = cross_val_score(model_final, X, y, cv=i)
    print( "i=" ,i , " " ,test_datascores.mean() )

i= 2   0.7519826674056532
i= 3   0.7800224466891134
i= 4   0.7901112996404476
i= 5   0.7900947837549432
i= 6   0.7900568353588486
i= 7   0.7889834083239595
i= 8   0.7878157175032175
i= 9   0.7890011223344557
i= 10   0.787852684144819
i= 11   0.7867564534231201
i= 12   0.7867417417417416
i= 13   0.7855105252803464
i= 14   0.7866886337868481
i= 15   0.7866290018832393
i= 16   0.7866680194805195
i= 17   0.7865832835311193
i= 18   0.7865759637188208
i= 19   0.7867228200009738
i= 20   0.7865656565656566
i= 21   0.7865052997943365
i= 22   0.7865576496674058
i= 23   0.7867452913219503
i= 24   0.7866880037932669
i= 25   0.7866666666666667
i= 26   0.7866839043309631
i= 27   0.7867564534231201
i= 28   0.7878744239631335
i= 29   0.7878383388950686


In [1054]:

param_grid = {
    'C': [0.1, 1, 10, 100, 1000],
    'penalty': ['l2']
}
for i in range (2,20):
    grid = GridSearchCV(estimator=model_final, param_grid=param_grid,scoring='accuracy',cv=i, n_jobs=-1,error_score='raise')
    grid.fit(X, y)

    # print the best parameters and accuracy score
    print(i , "Best parameters:", grid.best_params_)
    print("Accuracy:", grid.best_score_)

2 Best parameters: {'C': 10, 'penalty': 'l2'}
Accuracy: 0.7531062629112713
3 Best parameters: {'C': 10, 'penalty': 'l2'}
Accuracy: 0.7901234567901235
4 Best parameters: {'C': 10, 'penalty': 'l2'}
Accuracy: 0.7923534521068154
5 Best parameters: {'C': 10, 'penalty': 'l2'}
Accuracy: 0.7923419747661792
6 Best parameters: {'C': 1, 'penalty': 'l2'}
Accuracy: 0.7900568353588486
7 Best parameters: {'C': 10, 'penalty': 'l2'}
Accuracy: 0.7901082677165354
8 Best parameters: {'C': 100, 'penalty': 'l2'}
Accuracy: 0.7911739864864865
9 Best parameters: {'C': 10, 'penalty': 'l2'}
Accuracy: 0.7912457912457912
10 Best parameters: {'C': 100, 'penalty': 'l2'}
Accuracy: 0.790099875156055
11 Best parameters: {'C': 1, 'penalty': 'l2'}
Accuracy: 0.7867564534231201
12 Best parameters: {'C': 1000, 'penalty': 'l2'}
Accuracy: 0.7912162162162161
13 Best parameters: {'C': 10, 'penalty': 'l2'}
Accuracy: 0.7877401796839139
14 Best parameters: {'C': 10, 'penalty': 'l2'}
Accuracy: 0.788920776643991
15 Best parameters: 

In [1056]:
kf = KFold(n_splits=18)
for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model_final = LogisticRegression(C=100, penalty='l2')
        model_final.fit(X_train,y_train)

In [1057]:
test_data2 = test_data.drop(["Name","Ticket","Fare","Cabin","Parch","SibSp","PassengerId","Age","Embarked"], axis=1)

In [1058]:
df

,Survived,Pclass,Sex,Age_Adult,Age_Child,Age_Ederly,Age_teenager
0,0,3,0,1.0,0.0,0.0,0.0
1,1,1,1,1.0,0.0,0.0,0.0
2,1,3,1,1.0,0.0,0.0,0.0
3,1,1,1,1.0,0.0,0.0,0.0
4,0,3,0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
886,0,2,0,1.0,0.0,0.0,0.0
887,1,1,1,0.0,0.0,0.0,1.0
888,0,3,1,1.0,0.0,0.0,0.0
889,1,1,0,1.0,0.0,0.0,0.0


In [1059]:
df

,Survived,Pclass,Sex,Age_Adult,Age_Child,Age_Ederly,Age_teenager
0,0,3,0,1.0,0.0,0.0,0.0
1,1,1,1,1.0,0.0,0.0,0.0
2,1,3,1,1.0,0.0,0.0,0.0
3,1,1,1,1.0,0.0,0.0,0.0
4,0,3,0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
886,0,2,0,1.0,0.0,0.0,0.0
887,1,1,1,0.0,0.0,0.0,1.0
888,0,3,1,1.0,0.0,0.0,0.0
889,1,1,0,1.0,0.0,0.0,0.0


In [1060]:
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Adult,Age_Child,Age_Ederly,Age_teenager
0,892,3,"Kelly, Mr. James",0,Adult,0,0,330911,7.8292,NaN,Q,1.0,0.0,0.0,0.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,Adult,1,0,363272,7.0000,NaN,S,1.0,0.0,0.0,0.0
2,894,2,"Myles, Mr. Thomas Francis",0,Ederly,0,0,240276,9.6875,NaN,Q,0.0,0.0,1.0,0.0
3,895,3,"Wirz, Mr. Albert",0,Adult,0,0,315154,8.6625,NaN,S,1.0,0.0,0.0,0.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,Adult,1,1,3101298,12.2875,NaN,S,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",0,Adult,0,0,A.5. 3236,8.0500,C78,S,1.0,0.0,0.0,0.0
414,1306,1,"Oliva y Ocana, Dona. Fermina",1,Adult,0,0,PC 17758,108.9000,C105,C,1.0,0.0,0.0,0.0
415,1307,3,"Saether, Mr. Simon Sivertsen",0,Adult,0,0,SOTON/O.Q. 3101262,7.2500,C105,S,1.0,0.0,0.0,0.0
416,1308,3,"Ware, Mr. Frederick",0,Adult,0,0,359309,8.0500,C105,S,1.0,0.0,0.0,0.0


In [1061]:
predictions = model_final.predict(test_data2.values)

In [1062]:
final = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})

In [1063]:
final.to_csv('submission2.csv', index=False)